# Confiabilidade de viga mista

## Importando bibliotecas

In [43]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import import_ipynb
from math import sqrt

## Definindo funções

In [44]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    amostra: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(amostra):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [45]:
espessura_laje: float = 10 # BOTAR UNIDADE MARINA 
desvpad_espessura_laje: float = 0.06 * espessura_laje # mm

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [46]:
limite_escoamento_perfil: float = 250 * 1.08
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [47]:
limite_escoamento_armadura: float = 500 * 1.08
desvpad_escoamento_armadura: float = 0.08 * limite_escoamento_armadura

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [48]:
resistencia_concreto: float = 20 * 1.17
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [49]:
diametro_rebite: float = 1.59
desvpad_diametro_rebite: float = 0.01 * diametro_rebite

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [50]:
resistencia_rebite: float = 1.59
desvpad_resistencia_rebite: float = 0.01 * resistencia_rebite

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [51]:
carga_permanente_g1: float = 7.6
carga_permanente_g2: float = 5

variacao_carga: list = [3, 11]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9
espacamento_laje: float = 2.8

# Carga distrubuída do projeto
carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i] for i in range(len(carga_utilizacao_q2))]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * vao_laje ** 2) / 8 for i in range(len(carga_distribuida_projeto))]

altura_laje_ht: float = 45
coeficiente_ya1: float = 1.1
x: float = 10

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_laje_ht / 2) + espessuras_laje[j] - (x / 2))))
    index_matriz += 1
    
print(areas_minimas)


[[14.33003349862654, 13.677190872441711, 15.021338467419765, 16.321912590344333, 13.74382018988489, 13.912922702787952, 14.102276931587559, 14.01785168476603, 14.677553165226792, 13.407285383651818, 15.225446711299696, 15.360232247510885, 15.234317718724503, 13.879518836909151, 14.622811975171784, 12.913465041996526, 13.31624241900638, 14.800212988607376, 15.66511447467751, 13.41565140763893, 13.985122873584526, 13.466058884757398, 12.691527817725321, 14.586732608798481, 15.753555971162578, 13.679076817859443, 17.936022007419066, 15.516099739526094, 15.20244336493183, 13.468526217686971, 12.760098345086748, 16.318743636891053, 11.527494696917588, 13.310252163149705, 13.785117385181838, 13.195242364869273, 14.77696433187319, 13.521643883703328, 13.60221943906028, 15.357000602462959, 14.483017201048058, 15.030567254323339, 16.37994242268797, 14.70952552339386, 16.80429154368961, 15.088481722188396, 16.971083190298824, 13.972936650095303, 13.821191908446924, 12.983389845884194, 14.8567160

## Flambagem da seção

In [52]:
# Classificação da seção quanto a flambagem

altura_perfil: float = 404 # mm
espessura_perfil: float = 7.6 # mm
modulo_elasticidade_perfil: float = 200000 # MPa

esbeltez_alma: float = altura_perfil / espessura_perfil

limites_flambagem_local: list = []

def calculo_limite_flambagem_local(coef: float, modulo_elasticidade_perfil: float, limite_escoamento_perfil: float) -> float:
    return coef * sqrt(modulo_elasticidade_perfil / limite_escoamento_perfil)

countador_compacto: int = 0
countador_semicompacto: int = 0

for i in range(len(limites_escoamento_perfil)):
    limite = calculo_limite_flambagem_local(3.76, modulo_elasticidade_perfil, limites_escoamento_perfil[i])
    limites_flambagem_local.append(limite)
    
    if esbeltez_alma <= limite:
        countador_compacto += 1
    elif esbeltez_alma > limite and esbeltez_alma < calculo_limite_flambagem_local(5.7, modulo_elasticidade_perfil, limites_escoamento_perfil[i]):
        countador_semicompacto += 1
    else:
        print('Erro')
        
print(f'Compactas: {countador_compacto}')
print(f'Semi-compactas: {countador_semicompacto}')
        

Compactas: 1000
Semi-compactas: 0
